# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f90d33f7220>
├── 'a' --> tensor([[-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075]])
└── 'x' --> <FastTreeValue 0x7f902aeeda30>
    └── 'c' --> tensor([[ 0.7147,  0.5573,  0.3149,  0.4486],
                        [ 1.5436, -1.5523, -0.6522,  0.5021],
                        [ 1.7369,  1.3355, -0.3462,  0.2030]])

In [4]:
t.a

tensor([[-0.4568, -0.4701, -2.1553],
        [ 0.4081, -0.4990, -0.0075]])

In [5]:
%timeit t.a

61.3 ns ± 0.0193 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f90d33f7220>
├── 'a' --> tensor([[-1.6384,  0.6836, -0.1320],
│                   [ 0.9996,  1.1419,  0.5095]])
└── 'x' --> <FastTreeValue 0x7f902aeeda30>
    └── 'c' --> tensor([[ 0.7147,  0.5573,  0.3149,  0.4486],
                        [ 1.5436, -1.5523, -0.6522,  0.5021],
                        [ 1.7369,  1.3355, -0.3462,  0.2030]])

In [7]:
%timeit t.a = new_value

61.2 ns ± 0.0443 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075]]),
    x: Batch(
           c: tensor([[ 0.7147,  0.5573,  0.3149,  0.4486],
                      [ 1.5436, -1.5523, -0.6522,  0.5021],
                      [ 1.7369,  1.3355, -0.3462,  0.2030]]),
       ),
)

In [10]:
b.a

tensor([[-0.4568, -0.4701, -2.1553],
        [ 0.4081, -0.4990, -0.0075]])

In [11]:
%timeit b.a

57.2 ns ± 0.0148 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3080,  0.7063, -1.4383],
               [-0.2968,  1.4176,  0.5641]]),
    x: Batch(
           c: tensor([[ 0.7147,  0.5573,  0.3149,  0.4486],
                      [ 1.5436, -1.5523, -0.6522,  0.5021],
                      [ 1.7369,  1.3355, -0.3462,  0.2030]]),
       ),
)

In [13]:
%timeit b.a = new_value

476 ns ± 0.133 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

827 ns ± 0.0964 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 595 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 606 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f902066a4f0>
├── 'a' --> tensor([[[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]],
│           
│                   [[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]],
│           
│                   [[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]],
│           
│                   [[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]],
│           
│                   [[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]],
│           
│                   [[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]],
│           
│                   [[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]],
│           
│                   [[-0.4568, -0.4701, -2.1553],
│                    [ 0.4081, -0.4990, -0.0075]]])
└── 'x' --> <FastTreeValue 0x7f9019c64c40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 45 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9019634d00>
├── 'a' --> tensor([[-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075],
│                   [-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075],
│                   [-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075],
│                   [-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075],
│                   [-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075],
│                   [-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075],
│                   [-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075],
│                   [-0.4568, -0.4701, -2.1553],
│                   [ 0.4081, -0.4990, -0.0075]])
└── 'x' --> <FastTreeValue 0x7f902059eac0>
    └── 'c' --> tensor([[ 0.7147,  0.5573,  0.3149,  0.4486],
                        [ 1.5436, -1.5523, -0.6522,  0.5021],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 36.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.2 µs ± 97.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]],
       
               [[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]],
       
               [[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]],
       
               [[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]],
       
               [[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]],
       
               [[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]],
       
               [[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]],
       
               [[-0.4568, -0.4701, -2.1553],
                [ 0.4081, -0.4990, -0.0075]]]),
    x: Batch(
           c: tensor([[[ 0.7147,  0.5573,  0.3149,  0.4486],
                       [ 1.5436, -1.5523, -0.6522,  0.5021],
                       [ 1.7369,  1.3355, -0.3462,  0.2030]],
         

In [26]:
%timeit Batch.stack(batches)

79 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075],
               [-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075],
               [-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075],
               [-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075],
               [-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075],
               [-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075],
               [-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075],
               [-0.4568, -0.4701, -2.1553],
               [ 0.4081, -0.4990, -0.0075]]),
    x: Batch(
           c: tensor([[ 0.7147,  0.5573,  0.3149,  0.4486],
                      [ 1.5436, -1.5523, -0.6522,  0.5021],
                      [ 1.7369,  1.3355, -0.3462,  0.2030],
                      [ 0.7147,  0.5573,  0.3149,  0.4486],
                      [ 1.5436, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 298 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 526 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
